# <center> NFtorch 使用教程 </center>

## 一. 深度学习模型

### 1. 导入数据

In [1]:
from NFtorch.DeepLearning import Regression
import torch
from torch import nn
from sklearn import datasets

boston = datasets.load_boston()

X, y = boston.data, boston.target

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

from sklearn.preprocessing import StandardScaler

standard = StandardScaler()
standard.fit(X_train)
X_train_standard = standard.transform(X_train)
X_test_standard = standard.transform(X_test)

### 2. 构造模型

In [2]:
class module_net(nn.Module):
    def __init__(self, num_input, num_hidden, num_output):
        super(module_net, self).__init__()
        self.layer1 = nn.Linear(num_input, num_hidden)

        self.layer2 = nn.Tanh()

        self.layer3 = nn.Linear(num_hidden, num_output)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

## 3. 定义超参数

In [3]:
num_input = X_train_standard.shape[1]
num_hidden = 32
num_output = 1

### 4.实例化模型，实例化 *NFtorch* 中的 *Regression* 

In [4]:
net = module_net(num_input, num_hidden, num_output)

# 自定义参数, 这里采用的是 “学习率衰减” 的方法，如传入list，为梯度衰减，反之为正常lr。
reg = Regression(net, learning_rate=[1e-3, 1e-5, 1e-7], dropout=0.2, weight_decay=1e-6,
                     epoch=2000, batch_size=128) 

### 5.可选择性的设置函数

In [5]:
reg.set_device(device=0, use_more_gpu=True) # 设置使用哪块 GPU， 是否用多块 GPU 并行操作

# 这个是设置预处理的方法，默认为 None，保存模型用。可默认不设置
reg.set_methods(preprocessing_method='Standardazation',
                decomposition_method='PCA', save_path='DL_regression_save')

'''
设置 epoch 的中止装置：

功能：减少运算时间，充分利用资源。

原理：计算出最近的 10个 epoch 的 training losses 的均值 x，如果均值 x < limit，则进入下一个 learning rate
因此，这个中止装置需与 “学习率衰减” 装置搭配使用。

默认参数为： [1e-2, 1e-3, 1e-4]
'''
reg.set_limit(limit=[1e-2, 1e-3, 1e-4])

### 6.训练模型

In [6]:
reg.fit(X_train_standard, y_train, X_test_standard, y_test)

Now we're using the CPU
The error changes within 0.01
Training... epoch: 447, loss: 0.46830034255981445
 Testing... epoch: 447, loss: 19.718181610107422 !
Now learning rate is : 1e-05
The error changes within 0.001
Training... epoch: 467, loss: 0.4642278850078583
 Testing... epoch: 467, loss: 19.717761993408203 !
Now learning rate is : 1e-07
The error changes within 0.0001
Training... epoch: 487, loss: 0.4641951620578766
 Testing... epoch: 487, loss: 19.7177791595459 !
The meaning of the loop is not big, stop!!
Training completed!!! Time consuming: 1.1097571849822998


### 7.训练完毕，查看并保存 预测结果

In [7]:
reg.score()

mse: 19.717778678224963, rmse: 4.4404705469381245, mae: 2.86853988123875, mape: 11.750048585203873, r2: 0.8115522854411653, r2_adjusted: 0.783713418517701, rmsle: 0.15688453241565506


In [8]:
# 也可以使用 predict 函数获得 预测值
prediction = reg.predict(X_test_standard)

In [9]:
'''
保存结果：
1. losses（picutre，values）
2. Prediction（存放值，对应csv的count）
3. Picture（同对应csv的count）
3. 汇总的csv
4. weight：模型参数
'''
reg.save()

Save results success!
Save the value of prediction successfully!!
Save the picture successfully!


-----
## 分类模型同理， 调用 Classification，后续操作相同就不重复了

In [10]:
from NFtorch.DeepLearning import Classification